## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 24
country = "Denmark"; # Possible options: Portugal, Slovenia, Greece, Denmark, this if apply only to Slovenia and Denmark, because both countries have biomass

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 14:38:37


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.Pl = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.Php = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,3]))
model.Pb = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,4]))
model.Pimp = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,5]))
model.Ppv = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,6]))
model.Pwind = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,7]))
model.Phid = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,8]))
model.Pbio = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,9]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,9]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 0.95
model.m = 0.6
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Pgen = pyo.Var(model.t, domain =  pyo.NonNegativeReals, initialize = 0)
model.Pres = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Pex = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] >= m.EEVmax[ev]*m.m
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _RES_balance(m,t):
    return m.Pres[t] == m.Pwind[t] + m.Ppv[t] + m.Phid[t] + m.Pbio[t]
model.RES_balance = pyo.Constraint(model.t, rule = _RES_balance)

def _total_balance(m,t):
    #if (country == "Slovenia"):
        return m.Pgen[t] + m.Pres[t] + m.Pimp[t] - m.Pex[t] == m.Pl[t] + m.Pb[t] +sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
    #else:
        #return m.Pgen[t] + m.Pres[t] + m.Pimp[t] - m.Pex[t] ==  m.Pl[t] + m.Pb[t] + sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
model.total_balance = pyo.Constraint(model.t, rule = _total_balance)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
    #return m.EEVmin[ev] <= m.EEV[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)


## Objective function

In [11]:
def _FOag(m):
    return (sum(m.Pgen[t] for t in np.arange(1, n_time + 1))**2 + sum(m.Ppv[t] for t in np.arange(1, n_time + 1))**2)/ (sum(m.Ppv[t] for t in np.arange(1, n_time + 1))**2) + sum([m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2 for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmpge0qsyr6
  Lower bound: 588287589.7100073
  Upper bound: 588287589.7100073
  Number of objectives: 1
  Number of constraints: 917754
  Number of variables: 759553
  Number of nonzeros: 1829178
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 5.13
  Termination condition: optimal
  Termination message: Barrier - Optimal\x3a Objective = 5.8828758971e+08
  Error rc: 0
  Time: 11.08033561706543
# ----------------------------------------------------------
#   Solution Information
# -------------------

## Objective Function Value

In [13]:
pyo.value(model.FOag)

588287589.7106819

## End Time

In [14]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 14:41:49
Dif: 0:03:12


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
EEV_df = ext_pyomo_vals(model.EEV)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))

Charge cost: 11333735.454171581


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy()/1000000)))

Total Charge: 62.41315614423894


In [18]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,...,62336.679987,62829.415933,63493.062891,64243.467625,65019.814130,65820.496620,66572.149031,67149.409293,67677.785852,64943.200559
2,38103.743049,38103.743049,38103.743049,38103.743049,38103.743049,38103.743049,38103.743049,38103.743049,38103.743049,38103.743049,...,35454.438636,35920.333319,36543.942304,37254.479733,37988.468203,38747.405816,39471.145120,40031.462645,40538.056614,41053.794247
3,8169.093412,8238.652367,8308.352993,8378.202381,8448.219662,8518.434285,8589.019585,8659.860005,8730.970972,8802.383681,...,9158.404000,9158.404000,9158.404000,9158.404000,9158.404000,9158.404000,9158.404000,9158.404000,9158.404000,9243.363223
4,8169.183431,8238.833741,8308.625955,8378.567264,8448.676922,8518.984520,8589.663904,8660.599254,8731.806076,8803.315676,...,9148.570475,9148.570475,9148.570475,9148.570475,9148.570475,9148.570475,9148.570475,9148.570475,9148.570475,9222.436030
5,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,...,38341.564783,37120.370857,38555.720347,40827.547418,43545.963123,44624.528949,44904.242258,44026.120229,42985.576231,40952.534753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6325,45620.398633,46919.765569,48241.343696,49591.048357,50975.262659,52402.116075,53886.842900,55438.835012,57077.587435,58832.023009,...,60697.989964,56560.726280,52423.462595,48286.198911,48286.198911,44148.935226,40011.671542,40011.671542,35874.407857,44506.205455
6326,46905.573438,47013.419036,47121.307438,47229.332597,47337.535562,47445.967372,47554.862603,47664.113643,47773.787061,47883.971938,...,47528.734616,47528.734616,47528.734616,47528.734616,47528.734616,47528.734616,47528.734616,47528.734616,47528.734616,46849.256181
6327,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,...,55353.062060,55742.907219,56228.877289,56772.530044,57320.746915,57880.624792,58431.615553,58878.849176,59294.591159,57325.441806
6328,65569.031036,66490.011219,67417.055458,68351.431703,69294.290542,70247.056584,71212.765005,72192.021278,73187.018819,74200.382343,...,69392.070420,69392.070420,69392.070420,69392.070420,69392.070420,69392.070420,69392.070420,69392.070420,69392.070420,64833.138191


In [19]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,517.420831,518.669417,698.575745,789.899719,817.206848,842.823673,791.213064,607.642381,556.185852,621.010377
2,440.150939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,489.574662,490.415456,656.430511,747.934137,772.619442,798.881698,761.830846,589.807921,533.256809,542.881719
3,72.729907,73.219953,73.369081,73.525671,73.702401,73.910129,74.300316,74.568863,74.853650,75.171272,...,76.767868,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,95.361781
4,72.824664,73.316116,73.465488,73.622430,73.799641,74.007998,74.399352,74.668789,74.954550,75.273263,...,76.877414,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,95.546274
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,849.293635,854.576370,1510.894200,2391.396917,2861.490215,3275.376110,2434.478723,1215.704682,1044.734189,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6325,1284.630140,1367.754669,1391.134871,1420.741748,1457.067686,1501.950964,1562.870342,1633.675908,1725.002550,1846.774289,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13441.117138
6326,111.129934,113.521682,113.566739,113.710693,113.897859,114.138747,114.626559,115.001094,115.445703,115.984082,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,129.512463
6327,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,409.275772,410.363325,511.547443,572.266058,577.070390,589.345134,579.990275,470.772234,437.623141,477.549622
6328,935.822143,969.452824,975.836042,983.553942,992.482989,1002.911623,1016.535180,1030.796077,1047.365833,1066.698446,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1351.679128


In [20]:
PEV_df.sum()

1     2.265071e+06
2     2.070288e+06
3     2.111522e+06
4     2.151708e+06
5     2.193327e+06
6     2.263968e+06
7     2.329444e+06
8     2.411696e+06
9     2.508449e+06
10    2.668002e+06
11    2.121014e+06
12    1.657247e+06
13    1.903465e+06
14    2.343400e+06
15    1.772568e+06
16    1.694845e+06
17    2.558561e+06
18    3.279134e+06
19    3.618267e+06
20    3.920517e+06
21    3.313962e+06
22    2.147739e+06
23    3.900160e+06
24    5.208803e+06
dtype: float64

In [21]:
Pgen_df = ext_pyomo_vals(model.Pgen)

Pgen_df

,0
1,7.278585e+06
2,7.106935e+06
3,6.919984e+06
4,6.690223e+06
5,6.615490e+06
6,7.008578e+06
7,3.174561e+06
8,3.257149e+06
9,3.527561e+06
10,3.691071e+06


In [22]:
Pres_df = ext_pyomo_vals(model.Pres)

Pres_df

,0
1,5.202029e+06
2,5.864105e+06
3,5.383119e+06
4,4.117823e+06
5,3.434997e+06
6,4.004192e+06
7,5.191122e+06
8,4.282210e+06
9,4.371468e+06
10,3.906558e+06


In [23]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [24]:
Etriprelax_df.sum()

1          0.000021
2          0.000031
3          0.000031
4          0.000031
5          0.000031
6          0.000031
7          0.000031
8          0.000031
9          0.000031
10         0.000031
11         0.000031
12         0.000031
13      1074.470884
14      6533.501575
15      8350.704321
16     12251.114767
17     13671.367942
18     14283.455120
19     12660.400190
20     13295.113468
21     12328.740878
22      8840.625296
23      4714.849143
24    480151.085613
dtype: float64

In [25]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [26]:
Eminsocrelax_df.sum()

1       0.000000
2       0.000022
3       0.000032
4       0.000032
5       0.000032
6       0.000032
7       0.000032
8       0.000032
9       0.000032
10      0.000032
11      0.000032
12      0.000032
13      0.000032
14      0.000032
15      0.000032
16      0.000032
17      0.000032
18      0.000032
19      0.000032
20      0.000032
21      0.000032
22    132.155852
23      0.000032
24      0.000032
dtype: float64

In [27]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [28]:
Etripn_df.sum()

1     3.012639e+06
2     1.166828e+05
3     5.611484e+04
4     1.747803e+04
5     7.542035e+03
6     1.218573e+04
7     1.022651e+04
8     1.182157e+04
9     0.000000e+00
10    1.878932e+06
11    4.450396e+06
12    0.000000e+00
13    2.402219e+06
14    9.198751e+06
15    4.185460e+06
16    4.260775e+06
17    1.203396e+06
18    2.595200e+06
19    7.924992e+05
20    2.940490e+06
21    3.307096e+06
22    2.922840e+06
23    3.333886e+06
24    1.150694e+07
dtype: float64

In [29]:
Pex_df = ext_pyomo_vals(model.Pex)

In [30]:
Pex_df

,0
1,857.196086
2,967.404705
3,930.531307
4,895.481352
5,882.920013
6,913.738420
7,667.692029
8,666.044104
9,650.169063
10,653.945593


In [31]:
Pbio_df = ext_pyomo_vals(model.Pbio)

In [32]:
Pbio_df

,0
1,538066.6667
2,502133.3333
3,493577.7778
4,492488.8889
5,508977.7778
6,415955.5556
7,553000.0000
8,562800.0000
9,562800.0000
10,554555.5556


## Save results in csv files

In [33]:
import os 
folder = 'results_RESc_' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')

Pgen_df.to_csv(folder + '/Pgen.csv')
Pres_df.to_csv(folder + '/Pres.csv')
Pex_df.to_csv(folder + '/Pex.csv')